# Research Actions

This document is research of semantic similarity between including actions sentences from different documents. 

In [1]:
# import used modules
import os
import numpy as np
import time
import pickle
from scipy.linalg import block_diag

In [2]:
# import a model and a module for semantic analys
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('../model.bin', binary=True) 
model.init_sims(replace=True)

/home/ased/anaconda3/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [3]:
# import our module for semantic analys
import sys
sys.path.append('../')
import sem_analysis as sa


Loading the model...
Processing input...


## 1. Extract Sentences From Documents And Their Preprocessing

In [4]:
path = "Actions/"
files = list()
for i in os.listdir(path):
    if 'Sectionstext' in i:
        files.append(i)

In [5]:
sections = dict()
for j in files:
    f = open(path + j, 'r')
    new = list()
    for line in f:
        s = ''
        for i in line:
            if i != '\n':
                s = s + i
        if len(s) > 0:
            new.append(s)
    sections[j] = new
    f.close()

In [6]:
# We leave sentences only from documents of type '2_' (see types description in folder ../../Texts)
cur_sections = {i:sections[i] for i in sections.keys() if '2_' in i}

In current step we initialize a distance matrix for our sentences: an matrix element with indexes `i` and `j` is distance between `i`-th and `j`-th sentences. If sentences `i` and `j` from one document we set this element equal to $\infty$.

In [7]:
list_ = list()
keys = list(cur_sections.keys()).copy()
keys.sort()
for j in keys:
    if len(cur_sections[j]) > 0:
        list_.append(np.inf * np.ones((len(cur_sections[j]), len(cur_sections[j]))))
    else:
        cur_sections.pop(j)
D = block_diag(*np.array(list_))

In [8]:
phrases = list()
for j in keys:
    phrases = phrases + cur_sections[j]
print(len(phrases))

886


Now we transform our phrases to format for [RusVectores](https://rusvectores.org/ru/). See [tutorial for RusVectores](https://github.com/akutuzov/webvectors/blob/master/preprocessing/rusvectores_tutorial.ipynb).

In [9]:
if not 'action_data.pickle' in os.listdir():
    t = time.time()
    phrases_transform = list()
    for i in phrases:
        phrases_transform.append(sa.tag_ud(i))
        if len(phrases_transform) % 100 == 0:
            print(len(phrases_transform), time.time() - t)
    with open('action_data.pickle', 'wb') as f:
        pickle.dump(phrases_transform, f)
        f.close()
else:
    with open('action_data.pickle', 'rb') as f: 
        phrases_transform = pickle.load(f)
        f.close()

100 463.59754157066345
200 967.2637391090393
300 1478.5802619457245
400 1965.4721224308014
500 2476.349060535431
600 2974.829411506653
700 3440.2402532100677
800 3899.5513842105865


## 2. Similarity

### 2.1. Similarity withoud filtration

In [12]:
# Distance Matrix
import time
t = time.time()
for i in range(D.shape[0]):
    for j in range(i+1, D.shape[1]):
        if D[i, j] != np.inf:
            D[i,j] = model.wmdistance(phrases_transform[i], phrases_transform[j])
            D[j, i] = D[i, j]
            if i == 0 and j == 100:
                print((time.time() - t)/100*D.shape[0]*D.shape[0] * 0.9 * 0.5)

1626.7332340178489


#### Output

In [13]:
sim = dict()
for i in range(D.shape[0]):
    for j in range(i+1, D.shape[1]):
        if D[i,j] != np.inf and not np.isnan(D[i,j]):
            if not sim.__contains__(D[i,j]):
                sim[D[i,j]] = []
            sim[D[i,j]].append((i, j, phrases[i], phrases[j]))

In [14]:
keys = list(sim.keys())
keys.sort()

In [15]:
def print_eps(eps, keys, mydict):
    i = 0
    #while keys[i] > keys[0] - eps and i < len(keys):
    while keys[i] < eps and i < len(keys):
        if  keys[i] > 0:
            for item in mydict[keys[i]]:
                print(keys[i], item[0],':',item[2],'\t',item[1],':', item[3])
        i += 1

In [19]:
print_eps(0.6, keys, sim)

0.20382673936834655 412 : Также читайте : Как выбрать марку автомобиля  	 606 : Подробнее читайте , как выбрать марку автомобиля ; 
0.22344495100013712 412 : Также читайте : Как выбрать марку автомобиля  	 453 : Также читайте : Как выбрать автомобиль под себя 
0.22619180971547098 453 : Также читайте : Как выбрать автомобиль под себя  	 609 : Также читайте : Как выбрать автомобиль под свои потребности 
0.2319723895653806 412 : Также читайте : Как выбрать марку автомобиля  	 609 : Также читайте : Как выбрать автомобиль под свои потребности 
0.29917577929512224 418 : Также читайте : Какой пробег считается нормальным для б / у авто  	 615 : Также читайте : Какой пробег считается нормальным для подержанного автомобиля 
0.32565300043518336 10 : Это можно сделать на сайте ГИБДД или avtokod .  	 433 : Сделать это можно на сайте reestr - zalogov . 
0.4209312067353194 453 : Также читайте : Как выбрать автомобиль под себя  	 606 : Подробнее читайте , как выбрать марку автомобиля ; 
0.444167432008

### 2.2. SImilarity With Filtration

In [20]:
import pickle
def transform(phrase):
    def cond(word):
        _ = ['_NOUN', '_VERB','_ADV', '_ADJ']
        for i in _:
            if i in word:
                return True
        return False
    phrase = phrase.copy()
    for i in phrase:
        if not cond(i):
            phrase.remove(i)
    return phrase

with open('action_data.pickle', 'rb') as f: 
    phrases_transform = pickle.load(f)
new = list()
for i in phrases_transform:
    new.append(transform(i))

In [21]:
import time
t = time.time()
for i in range(D.shape[0]):
    for j in range(i+1, D.shape[1]):
        if D[i, j] != np.inf:
            D[i,j] = model.wmdistance(new[i], new[j])
            D[j, i] = D[i, j]
            if i == 0 and j == 100:
                print((time.time() - t)/100*D.shape[0]*D.shape[0] * 0.9 * 0.5)

598.0595893993378


#### Output

In [23]:
sim_filtr = dict()
for i in range(D.shape[0]):
    for j in range(i+1, D.shape[1]):
        if D[i,j] != np.inf and not np.isnan(D[i,j]):
            if not sim_filtr.__contains__(D[i,j]):
                sim_filtr[D[i,j]] = []
            sim_filtr[D[i,j]].append((i, j, phrases[i], phrases[j]))

In [24]:
keys = list(sim_filtr.keys())
keys.sort()

In [25]:
def print_eps(eps, keys, mydict):
    i = 0
    while keys[i] < eps and i < len(keys):
        if  keys[i] > 0:
            for item in mydict[keys[i]]:
                print(keys[i], item[0],':',item[2],'\t',item[1],':', item[3])
        i += 1

In [26]:
print_eps(0.6, keys, sim_filtr)

0.20382673936834655 412 : Также читайте : Как выбрать марку автомобиля  	 606 : Подробнее читайте , как выбрать марку автомобиля ; 
0.22344495100013712 412 : Также читайте : Как выбрать марку автомобиля  	 453 : Также читайте : Как выбрать автомобиль под себя 
0.22619180971547098 453 : Также читайте : Как выбрать автомобиль под себя  	 609 : Также читайте : Как выбрать автомобиль под свои потребности 
0.2319723895653806 412 : Также читайте : Как выбрать марку автомобиля  	 609 : Также читайте : Как выбрать автомобиль под свои потребности 
0.27626750830583574 487 : Также читайте : Как самостоятельно проверить тех .  	 512 : Также читайте : Как проверить ПТС на подлинность 
0.29917577929512224 418 : Также читайте : Какой пробег считается нормальным для б / у авто  	 615 : Также читайте : Какой пробег считается нормальным для подержанного автомобиля 
0.4209312067353194 453 : Также читайте : Как выбрать автомобиль под себя  	 606 : Подробнее читайте , как выбрать марку автомобиля ; 
0.4720